In [1]:
import tensorflow as tf
import numpy as np
import pickle
import os

from tensorflow import keras
from tensorflow.keras import layers

tf.keras.backend.clear_session()

In [ ]:
time_steps=30
no_of_bins=32
no_of_bands=11
no_of_soil_features=36

In [ ]:

input =layers.Input(shape=(1,32,11))

x=layers.Conv2D(32,(1,2), activation='relu',input_shape=(1,32,11),padding='same')(input)
x=layers.BatchNormalization()(x)
x=layers.MaxPool2D((1,2))(x)
x=layers.Conv2D(64,(1,2),activation='relu',padding='same')(x)
x=layers.BatchNormalization()(x)
x=layers.MaxPool2D((1,2))(x)
x=layers.Flatten()(x)
x=layers.BatchNormalization()(x)
cnn_keras = keras.Model(inputs=input, outputs=x)


In [ ]:
keras.utils.plot_model(cnn_keras, "cnn_keras.png", show_shapes=True)

In [ ]:
# define two sets of inputs
inputA = layers.Input(shape=(time_steps,1,32,11), name="time_series")
inputB = layers.Input(shape=(1,32,36), name="soil")

# the first branch operates on the first input
x = layers.TimeDistributed(cnn_keras)(inputA)
x = layers.LSTM(256,return_sequences=True)(x)
x = layers.Dense(64)(x)
x = layers.Flatten()(x)
x = keras.Model(inputs=inputA, outputs=x)

# the second branch opreates on the second input
y = layers.Conv2D(48,(1,2), activation='relu',input_shape=(1,32,36),padding='same')(inputB)#32
y = layers.BatchNormalization()(y)
y = layers.MaxPool2D((1,2))(y)#though this is not mentioned in paper
y = layers.Conv2D(64,(1,2),activation='relu',padding='same')(y)
y = layers.BatchNormalization()(y)
y = layers.MaxPool2D((1,2))(y)
y = layers.Flatten()(y)
y = keras.Model(inputs=inputB, outputs=y)
# combine the output of the two branches
combined = layers.concatenate([x.output, y.output])
# print(x.output.shape)
# print(y.output.shape)
# apply a FC layer and then a regression prediction on the
# combined outputs
z = layers.Dense(64)(combined)     
z = layers.Dropout(0.5)(z)
z = layers.Dense(1)(z)
# our model will accept the inputs of the two branches and
# then output a single value
model = keras.Model(inputs=[x.input, y.input], outputs=z)

In [ ]:
keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)

In [ ]:
# Hyperparameters
epochs=200
batch_size=16
learning_rate = 2*1e-3 

In [ ]:
opt=keras.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
req_indices=data['output_year']==2015
# test_indices=[i in range(test_indices) and test_indices[i]==True]
test_indices=[]
train_indices=[]
for i in range(0, len(req_indices)):
    if req_indices[i]==True:
        test_indices.append(i)
    else:
        train_indices.append(i)
print(len(test_indices))
print(len(train_indices))


In [ ]:
test_out_img_composite=out_img_composite[test_indices,:,:,:,:]
test_out_soil=out_soil[test_indices,:,:,:]
test_yield=data['output_yield'][test_indices]


train_out_img_composite=out_img_composite[train_indices,:,:,:,:]
train_out_soil=out_soil[train_indices,:,:,:]
train_yield=data['output_yield'][train_indices]

In [ ]:
# model.compile(optimizer='adam', loss='mse')
# history=model.fit({"time_series": a1, "soil": a2}, b, batch_size=batch_size, epochs=epochs)
model.compile(optimizer=opt, loss='mse')
history=model.fit([train_out_img_composite,train_out_soil], train_yield, batch_size=batch_size, epochs=epochs)

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate([test_out_img_composite,test_out_soil], test_yield, batch_size=batch_size)
print("test loss, test acc:", results)

In [ ]:
print("Generate predictions for test samples")
predictions = model.predict([test_out_img_composite,test_out_soil])
# print("predictions shape:", predictions.shape)
predictions=predictions.reshape((1,predictions.shape[0]))
#print(predictions)
#print(test_yield)

In [ ]:
mape=np.array((abs(predictions-test_yield)/test_yield)*100)
print(np.mean(mape))    

In [ ]:
rmse=np.sqrt(np.mean((predictions-test_yield)**2))
print(rmse)

In [ ]:
# !pip install pyyaml h5py  # Required to save models in HDF5 format
model_name=str(len(global_processed_files))+'_opt_'+str(epochs)+'_'+str(batch_size)+'_'+str(int(learning_rate*1000))+'_lstm_256_64_cnnsoil_48_64'
print(model_name)
# model.save("/content/drive/MyDrive/data_Lakshya_op/"+model_name+".h5")
model.save(model_name+'.h5')

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
#import torch
from bs4 import BeautifulSoup
from pathlib import Path
import matplotlib as mpl
import matplotlib.pyplot as plt


def plot_county_errors(data, test_yield, predictions, svg_file=Path('/content/drive/MyDrive/data_Lakshya_op/counties.svg'), save_colorbar=True):
    """
    For the most part, reformatting of
    https://github.com/JiaxuanYou/crop_yield_prediction/blob/master/6%20result_analysis/yield_map.py

    Generates an svg of the counties, coloured by their prediction error.

    Parameters
    ----------
    model: pathlib Path
        Path to the model being plotted.
    svg_file: pathlib Path, default=Path('data/counties.svg')
        Path to the counties svg file used as a base
    save_colorbar: boolean, default=True
        Whether to save a colorbar too.
    """

   # model_sd = torch.load(model, map_location='cpu')

    #model_dir = model.parents[0]

    real_values = test_yield
    pred_values = predictions

    #gp = True
    #try:
    #    gp_values = model_sd['test_pred_gp']
    #except KeyError:
    gp = False

    indices =  data['output_index'][test_indices]                     #model_sd['test_indices']]

    pred_err = 100 * abs(pred_values - real_values)/real_values
    pred_dict = {}

    pred_err = np.transpose(pred_err, axes = (1,0))

    #return zip(indices, pred_err)
    for idx, err in zip(indices, pred_err):
        state, county = idx

        state = str(state).zfill(2)
        county = str(county).zfill(3)

        pred_dict[state + county] = err

    print(len(pred_dict))

    #model_info = model.name[:-8].split('_')

    colors = ["#7dcea0","#52be80","#27ae60","#229954", "#1e8449", "#196f3d", "#145a32", "#17202a"]#aditya
    # colors = ["#b2182b", "#d6604d", "#f4a582", "#fddbc7", "#d1e5f0", "#92c5de", "#4393c3", "#2166ac"]#original
    # colors = ["#2166ac", "#4393c3", "#92c5de", "#d1e5f0",  "#fddbc7",  "#f4a582",  "#d6604d", "#b2182b"]#rev
    # colors = ["#2166ac", "#92c5de", "#d1e5f0",  "#f4a582",  "#f4a582",  "#d6604d", "#b2182b"]
    


    _single_plot(pred_dict, svg_file, '/content/drive/MyDrive/data_Lakshya_op/'+model_name+'.svg', colors)

    if gp:
        gp_pred_err = gp_values - real_values
        gp_dict = {}
        for idx, err in zip(indices, gp_pred_err):
            state, county = idx

            state = str(state).zfill(2)
            county = str(county).zfill(3)

            gp_dict[state + county] = err

        _single_plot(gp_dict, svg_file, model_dir / f'{model_info[0]}_{model_info[1]}_gp.svg', colors)

    if save_colorbar:
        _save_colorbar('/content/drive/MyDrive/data_Lakshya_op/colorbar.png', colors)


def _single_plot(err_dict, svg_file, savepath, colors):

    # load the svg file
    svg = open(svg_file,'r').read()
    # Load into Beautiful Soup
    soup = BeautifulSoup(svg, features="html.parser")
    # Find counties
    paths = soup.findAll('path')

    path_style = 'font-size:12px;fill-rule:nonzero;stroke:#FFFFFF;stroke-opacity:1;stroke-width:0.1' \
                 ';stroke-miterlimit:4;stroke-dasharray:none;stroke-linecap:butt;marker-start' \
                 ':none;stroke-linejoin:bevel;fill:'

    for p in paths:
        if p['id'] not in ["State_Lines", "separator"]:
            try:
                rate = err_dict[p['id']]
            except KeyError:
                continue
            if rate > 70:
                color_class = 6
            elif rate > 50:
                color_class = 5
            elif rate > 40:
                color_class = 4
            elif rate > 30:
                color_class = 3
            elif rate > 20:
                color_class = 2
            elif rate > 10:
                color_class = 1
            elif rate > 0:
                color_class = 0
            else:
                color_class = 0

            color = colors[color_class]
            p['style'] = path_style + color
    soup = soup.prettify()
    with open(savepath, 'w') as f:
        f.write(soup)


def _save_colorbar(savedir, colors):
    fig = plt.figure()
    ax = fig.add_axes([0.1, 0.1, 0.02, 0.8])

    cmap = mpl.colors.ListedColormap(colors[1:-1])

    cmap.set_over(colors[-1])
    cmap.set_under(colors[0])

    bounds = [0, 10, 20, 30, 40, 50, 60, 70]

    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
    cb = mpl.colorbar.ColorbarBase(ax, cmap=cmap,
                                   norm=norm,
                                   # to use 'extend', you must
                                   # specify two extra boundaries:
                                   boundaries=[0] + bounds + [70],
                                   extend='both',
                                   ticks=bounds,  # optional
                                   spacing='proportional',
                                   orientation='vertical')
    plt.savefig(savedir, dpi=300, bbox_inches='tight')


In [ ]:
a = plot_county_errors(data, test_yield, predictions, svg_file=Path('/content/drive/MyDrive/data_Lakshya_op/counties.svg'), save_colorbar=True)